Script to analyse data related to fisheries. These data concerns:

* Target 14.4 (FMSY/F and B/BMSY)
* Target 14.6 (TAC/Catch)
* Target 14.a (SAD/TAC) 

Data comes from several sources:

For all indicators, we need data on Catches by Country (ICES Official Catches data)

Additionally, for each indicator we need:

* FMSY/F and B/BMSY: Stock Assessment data (ICES)
* TAC/Catch: TAC data (Carpenter or, alternatively, from EC PDFs)
* SAD/TAC: TAC (Carpenter or, alternatively, from OJ PDFs) and SAD (ICES data or Carpenter)

If we want to get all indicators with the same data, then merging procedure is: 

Catches by Country <-> Stock Assessment <-> TAC <-> SAD

If we calculate indicators separately, we can do three merges and get:
* Catches by Country <-> Stock Assessment: FMSY/F and B/BMSY
* Catches by Country <-> TAC: TAC/Catch
* Catches by Country <-> TAC <-> SAD: SAD/TAC

Notes from Rudi's meeting
1. Difference between officialNominalCatces and catches from StockAssesment dataset. For what stock do we have large differences? <10% would be alright. 
2. Stock Assesment is not done every year. Rudi: Do they provide a trend for those years that they don't assess? It's odd to use different years for different stocks, but it's possible. 
3. BMSY in ICES. Btrigger Bpa is the same as Btrigger, go for that one. 
4. Effort is used because it's hard to track the catches. Send questions to Rudi


In [406]:
import os
import pandas as pd
import numpy as np

In [3]:
pd.set_option('display.max_columns', 500)

In [4]:
countries=['Belgium','Bulgaria','Cyprus', 'Greece','Germany','Croatia','Italy', 
           'Denmark','Estonia','Spain','Finland','France','Ireland','Lithuania',
           'Latvia','Malta','Netherlands','Poland','Portugal', 'Romania',
           'Sweden','United Kingdom of Great Britain and Northern Ireland', "United Kingdom of GB"]

In [5]:
country_to_abbrev = {
    "Andorra": "AD",
    "United Arab Emirates": "AE",
    "Afghanistan": "AF",
    "Antigua and Barbuda": "AG",
    "Anguilla": "AI",
    "Albania": "AL",
    "Armenia": "AM",
    "Angola": "AO",
    "Antarctica": "AQ",
    "Argentina": "AR",
    "American Samoa": "AS",
    "Austria": "AT",
    "Australia": "AU",
    "Aruba": "AW",
    "Åland Islands": "AX",
    "Azerbaijan": "AZ",
    "Bosnia and Herzegovina": "BA",
    "Barbados": "BB",
    "Bangladesh": "BD",
    "Belgium": "BE",
    "Burkina Faso": "BF",
    "Bulgaria": "BG",
    "Bahrain": "BH",
    "Burundi": "BI",
    "Benin": "BJ",
    "Saint Barthélemy": "BL",
    "Bermuda": "BM",
    "Brunei Darussalam": "BN",
    "Bolivia (Plurinational State of)": "BO",
    "Bonaire, Sint Eustatius and Saba": "BQ",
    "Brazil": "BR",
    "Bahamas": "BS",
    "Bhutan": "BT",
    "Bouvet Island": "BV",
    "Botswana": "BW",
    "Belarus": "BY",
    "Belize": "BZ",
    "Canada": "CA",
    "Cocos (Keeling) Islands": "CC",
    "Congo, Democratic Republic of the": "CD",
    "Central African Republic": "CF",
    "Congo": "CG",
    "Switzerland": "CH",
    "Côte d'Ivoire": "CI",
    "Cook Islands": "CK",
    "Chile": "CL",
    "Cameroon": "CM",
    "China": "CN",
    "Colombia": "CO",
    "Costa Rica": "CR",
    "Cuba": "CU",
    "Cabo Verde": "CV",
    "Curaçao": "CW",
    "Christmas Island": "CX",
    "Cyprus": "CY",
    "Czechia": "CZ",
    "Germany": "DE",
    "Djibouti": "DJ",
    "Denmark": "DK",
    "Dominica": "DM",
    "Dominican Republic": "DO",
    "Algeria": "DZ",
    "Ecuador": "EC",
    "Estonia": "EE",
    "Egypt": "EG",
    "Western Sahara": "EH",
    "Eritrea": "ER",
    "Spain": "ES",
    "Ethiopia": "ET",
    "Finland": "FI",
    "Fiji": "FJ",
    "Falkland Islands (Malvinas)": "FK",
    "Micronesia (Federated States of)": "FM",
    "Faroe Islands": "FO",
    "France": "FR",
    "Gabon": "GA",
    "United Kingdom of Great Britain and Northern Ireland": "UK", #original is GB, Eurostat uses UK
    "United Kingdom of GB": "GB",
    "Grenada": "GD",
    "Georgia": "GE",
    "French Guiana": "GF",
    "Guernsey": "GG",
    "Ghana": "GH",
    "Gibraltar": "GI",
    "Greenland": "GL",
    "Gambia": "GM",
    "Guinea": "GN",
    "Guadeloupe": "GP",
    "Equatorial Guinea": "GQ",
    "Greece": "EL", #original ir GR, Eurostat uses EL
    "South Georgia and the South Sandwich Islands": "GS",
    "Guatemala": "GT",
    "Guam": "GU",
    "Guinea-Bissau": "GW",
    "Guyana": "GY",
    "Hong Kong": "HK",
    "Heard Island and McDonald Islands": "HM",
    "Honduras": "HN",
    "Croatia": "HR",
    "Haiti": "HT",
    "Hungary": "HU",
    "Indonesia": "ID",
    "Ireland": "IE",
    "Israel": "IL",
    "Isle of Man": "IM",
    "India": "IN",
    "British Indian Ocean Territory": "IO",
    "Iraq": "IQ",
    "Iran (Islamic Republic of)": "IR",
    "Iceland": "IS",
    "Italy": "IT",
    "Jersey": "JE",
    "Jamaica": "JM",
    "Jordan": "JO",
    "Japan": "JP",
    "Kenya": "KE",
    "Kyrgyzstan": "KG",
    "Cambodia": "KH",
    "Kiribati": "KI",
    "Comoros": "KM",
    "Saint Kitts and Nevis": "KN",
    "Korea (Democratic People's Republic of)": "KP",
    "Korea, Republic of": "KR",
    "Kuwait": "KW",
    "Cayman Islands": "KY",
    "Kazakhstan": "KZ",
    "Lao People's Democratic Republic": "LA",
    "Lebanon": "LB",
    "Saint Lucia": "LC",
    "Liechtenstein": "LI",
    "Sri Lanka": "LK",
    "Liberia": "LR",
    "Lesotho": "LS",
    "Lithuania": "LT",
    "Luxembourg": "LU",
    "Latvia": "LV",
    "Libya": "LY",
    "Morocco": "MA",
    "Monaco": "MC",
    "Moldova, Republic of": "MD",
    "Montenegro": "ME",
    "Saint Martin (French part)": "MF",
    "Madagascar": "MG",
    "Marshall Islands": "MH",
    "North Macedonia": "MK",
    "Mali": "ML",
    "Myanmar": "MM",
    "Mongolia": "MN",
    "Macao": "MO",
    "Northern Mariana Islands": "MP",
    "Martinique": "MQ",
    "Mauritania": "MR",
    "Montserrat": "MS",
    "Malta": "MT",
    "Mauritius": "MU",
    "Maldives": "MV",
    "Malawi": "MW",
    "Mexico": "MX",
    "Malaysia": "MY",
    "Mozambique": "MZ",
    "Namibia": "NA",
    "New Caledonia": "NC",
    "Niger": "NE",
    "Norfolk Island": "NF",
    "Nigeria": "NG",
    "Nicaragua": "NI",
    "Netherlands": "NL",
    "Norway": "NO",
    "Nepal": "NP",
    "Nauru": "NR",
    "Niue": "NU",
    "New Zealand": "NZ",
    "Oman": "OM",
    "Panama": "PA",
    "Peru": "PE",
    "French Polynesia": "PF",
    "Papua New Guinea": "PG",
    "Philippines": "PH",
    "Pakistan": "PK",
    "Poland": "PL",
    "Saint Pierre and Miquelon": "PM",
    "Pitcairn": "PN",
    "Puerto Rico": "PR",
    "Palestine, State of": "PS",
    "Portugal": "PT",
    "Palau": "PW",
    "Paraguay": "PY",
    "Qatar": "QA",
    "Réunion": "RE",
    "Romania": "RO",
    "Serbia": "RS",
    "Russian Federation": "RU",
    "Rwanda": "RW",
    "Saudi Arabia": "SA",
    "Solomon Islands": "SB",
    "Seychelles": "SC",
    "Sudan": "SD",
    "Sweden": "SE",
    "Singapore": "SG",
    "Saint Helena, Ascension and Tristan da Cunha": "SH",
    "Slovenia": "SI",
    "Svalbard and Jan Mayen": "SJ",
    "Slovakia": "SK",
    "Sierra Leone": "SL",
    "San Marino": "SM",
    "Senegal": "SN",
    "Somalia": "SO",
    "Suriname": "SR",
    "South Sudan": "SS",
    "Sao Tome and Principe": "ST",
    "El Salvador": "SV",
    "Sint Maarten (Dutch part)": "SX",
    "Syrian Arab Republic": "SY",
    "Eswatini": "SZ",
    "Turks and Caicos Islands": "TC",
    "Chad": "TD",
    "French Southern Territories": "TF",
    "Togo": "TG",
    "Thailand": "TH",
    "Tajikistan": "TJ",
    "Tokelau": "TK",
    "Timor-Leste": "TL",
    "Turkmenistan": "TM",
    "Tunisia": "TN",
    "Tonga": "TO",
    "Turkey": "TR",
    "Trinidad and Tobago": "TT",
    "Tuvalu": "TV",
    "Taiwan, Province of China": "TW",
    "Tanzania, United Republic of": "TZ",
    "Ukraine": "UA",
    "Uganda": "UG",
    "United States Minor Outlying Islands": "UM",
    "United States of America": "US",
    "Uruguay": "UY",
    "Uzbekistan": "UZ",
    "Holy See": "VA",
    "Saint Vincent and the Grenadines": "VC",
    "Venezuela (Bolivarian Republic of)": "VE",
    "Virgin Islands (British)": "VG",
    "Virgin Islands (U.S.)": "VI",
    "Viet Nam": "VN",
    "Vanuatu": "VU",
    "Wallis and Futuna": "WF",
    "Samoa": "WS",
    "Yemen": "YE",
    "Mayotte": "YT",
    "South Africa": "ZA",
    "Zambia": "ZM",
    "Zimbabwe": "ZW",
}
    
# invert the dictionary
abbrev_to_country = dict(map(reversed, country_to_abbrev.items()))

## Data manipulation

In [1062]:
# from ICES_Indicators excel file
stockW = ['cod.27.47d20',
 'bss.27.4bc7ad-h',
 'ghl.27.561214',
 'hke.27.8c9a',
 'meg.27.7b-k8abd',
 'hke.27.3a46-8abd',
 'mac.27.nea',
 'san.sa.3r',
 'sol.27.7d',
 'whg.27.47d',
 'pok.27.1-2',
 'cod.2127.1f14',
 'sol.27.20-24',
 'cod.21.1',
 'her.27.3a47d',
 'nop.27.3a4',
 'cod.27.7e-k',
 'lin.27.5a',
 'her.27.irls',
 'her.27.25-2932',
 'ple.27.420',
 'had.27.7b-k',
 'ple.27.7d',
 'ple.27.7a',
 'spr.27.4',
 'had.27.7a',
 'spr.27.22-32',
 'had.27.5a',
 'san.sa.1r',
 'her.27.3031',
 'reb.27.1-21',
 'pok.27.3a46',
 'her.27.1-24a514a',
 'sol.27.8ab',
 'mon.27.78abd',
 'pok.27.6',
 'bss.27.8ab',
 'sol.27.7e',
 'lez.27.4a6a',
 'reg.27.1-2',
 'ldb.27.8c9a',
 'san.sa.4',
 'whg.27.6a',
 'cod.27.21',
 'cod.27.7a',
 'had.27.46a20',
 'lez.27.6b',
 'hom.27.2a4a5b6a7a-ce-k8',
 'ple.27.21-23',
 'cod.27.5a',
 'cap.27.1-2',
 'cod.27.22-24',
 'pok.27.5a',
 'cod.27.1-2',
 'whg.27.7b-ce-k',
 'pra.27.3a4a',
 'mon.27.8c9a',
 'reb.27.1-2',
 'sol.27.7fg',
 'whb.27.1-91214',
 'usk.27.5a14',
 'her.27.5a',
 'her.27.6a7bc',
 'had.27.6b',
 'sol.27.4',
 'san.sa.2r',
 'had.27.1-2',
 'her.27.28',
 'hom.27.9a',
 'sol.27.7a',
 'bli.27.5b67',
 'reg.27.561214',
 'meg.27.8c9a',
 'cod.27.6a',
 'her.27.20-24',
 'pra.27.1-2',
 'ple.27.7fg',
 'her.27.nirs'
 ]


In [6]:
# https://neweconomics.org/campaigns/landing-the-blame
pd.ExcelFile(("../data/icesTACcomparison.xlsx")).sheet_names

['Menus',
 'Table of contents',
 'ICES advice',
 'Council agreed TAC',
 'Comparison',
 'Table for results',
 'Overall results',
 'Results by Member State',
 'Sea basin',
 'Third country',
 'Results by % difference',
 'Results by # of TACs',
 'Results by # of TACs by MS',
 'Results by species',
 'ID',
 'Matching - ICES-TAC',
 'Matching - Final TACs',
 'Matching - EU share',
 'Matching - TAC split share',
 'Matching - ICES area share']

In [7]:
pd.read_excel(("../data/icesTACcomparison.xlsx"), 1)

,Tab,Description
0,ICES advice,ICES advice by TAC and year
1,Council agreed TAC,Council agreed TAC by Member State and year
2,Comparison,Comparing ICES advice and agreed TACs by Membe...
3,Overall results,Calculates the difference between TACs and ICE...
4,Results by Member State,Calculates the difference between TACs and ICE...
5,Results by % difference,Calculates the difference between TACs and ICE...
6,Results by # of TACs,Calculates the number of TACs that exceed ICES...
7,Results by # of TACs by MS,Calculates the number of TACs that exceed ICES...
8,Results by third country share,Calculates the difference between TACs and ICE...
9,Results by species,Calculates the difference between TACs and ICE...


### StockAssessment

In [1414]:
# https://standardgraphs.ices.dk/stockList.aspx
stockList = []
stockL = ['stockAssesment2020','stockAssesment2021', 'stockAssesment2022']
for i in stockL:
    stock = pd.read_csv(("../data/{}/StockAssessment.csv".format(i)), names=range(138))
    stock.columns = stock.iloc[0,:]
    stock = stock[1:]
    stock['enName'] = stock["StockDescription"].str.extract(r"^(.+?) ?(?:\d|\(|$)" , expand=False)
    stock['speciesAcronym'] = stock["FishStock"].str.extract(r"^([^.]*).*" , expand=False)
    stock = stock.loc[:,['Year','enName', 'speciesAcronym','FishStock','StockKey', 'SpeciesName', "ICES Areas (splited with character '~')", 
                     'StockSize', 'StockSizeDescription', 'StockSizeUnits',
                      'FishingPressure', 'FishingPressureDescription', 'FishingPressureUnits',
                     'Flim', 'Fpa', 'Blim', 'Bpa', 'FMSY', 'MSYBtrigger', 
                     'CatchesLadingsUnits', 'Landings', 'OfficialLandings', 'Catches',
                     'Report', 'AssessmentKey','AssessmentYear']]
    stock.Year = stock.Year.astype(int)
    stock = stock[stock.Year.isin([2012,2016,2020])]
    stockList.append(stock)

C:\Users\isaldiviagonzatti\AppData\Local\Temp\ipykernel_31160\3454832697.py:5: DtypeWarning: Columns (0,1,3,4,6,10,12,13,14,17,18,19,20,21,22,23,27,28,29,30,31,32,33,34,35,36,39,40,41,42,43,44,45,46,47,48,49,50,51,52,56,57,58,59,60,62,65,66,67,68,69,70,71,72,73,74,75,76,79,82,85,88,91,94,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135) have mixed types. Specify dtype option on import or set low_memory=False.
  stock = pd.read_csv(("../data/{}/StockAssessment.csv".format(i)), names=range(138))
C:\Users\isaldiviagonzatti\AppData\Local\Temp\ipykernel_31160\3454832697.py:5: DtypeWarning: Columns (0,1,3,4,6,10,12,13,14,17,18,19,20,21,22,23,27,28,29,30,31,32,33,34,35,36,39,40,41,42,43,44,45,46,47,48,49,50,51,52,54,56,57,58,59,60,62,63,65,66,68,69,70,71,72,73,74,75,76,79,82,85,88,91,94,97,100,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,12

In [1421]:
stockList[0]

,Year,enName,speciesAcronym,FishStock,StockKey,SpeciesName,ICES Areas (splited with character '~'),StockSize,StockSizeDescription,StockSizeUnits,FishingPressure,FishingPressureDescription,FishingPressureUnits,Flim,Fpa,Blim,Bpa,FMSY,MSYBtrigger,CatchesLadingsUnits,Landings,OfficialLandings,Catches,Report,AssessmentKey,AssessmentYear
27,2012,Sandeel,san,san.sa.3r,169248,Ammodytes,27.3.a ~ 27.4.a ~ 27.4.b,150242,SSB,tonnes,0.125,F,Year-1,NaN,NaN,80000,129000,NaN,NaN,tonnes,40116,NaN,40116,https://doi.org/10.17895/ices.advice.5762,13298,2020
31,2016,Sandeel,san,san.sa.3r,169248,Ammodytes,27.3.a ~ 27.4.a ~ 27.4.b,232815,SSB,tonnes,0.125,F,Year-1,NaN,NaN,80000,129000,NaN,NaN,tonnes,42845,NaN,42845,https://doi.org/10.17895/ices.advice.5762,13298,2020
35,2020,Sandeel,san,san.sa.3r,169248,Ammodytes,27.3.a ~ 27.4.a ~ 27.4.b,221239,SSB,tonnes,NaN,F,Year-1,NaN,NaN,80000,129000,NaN,NaN,tonnes,NaN,NaN,NaN,https://doi.org/10.17895/ices.advice.5762,13298,2020
65,2012,Sandeel,san,san.sa.2r,169247,Ammodytes,27.4.b ~ 27.4.c,45982,SSB,tonnes,0.14,F,Year-1,NaN,NaN,56000,84000,NaN,NaN,tonnes,10594,NaN,10594,https://doi.org/10.17895/ices.advice.5761,13301,2020
69,2016,Sandeel,san,san.sa.2r,169247,Ammodytes,27.4.b ~ 27.4.c,31351,SSB,tonnes,0.172,F,Year-1,NaN,NaN,56000,84000,NaN,NaN,tonnes,5230,NaN,5230,https://doi.org/10.17895/ices.advice.5761,13301,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7188,2016,Beaked redfish,reb,reb.27.14b,169265,Sebastes mentella,27.14.b.1 ~ 27.14.b.2,1.013692,Stock Size: Relative,NaN,NaN,Fishing pressure: Relative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tonnes,3061.0,NaN,NaN,https://doi.org/10.17895/ices.advice.5847,13332,2020
7226,2012,Beaked redfish,reb,reb.27.5a14,169263,Sebastes mentella,27.14.a ~ 27.14.b.1 ~ 27.14.b.2 ~ 27.5.a.1 ~ 2...,0.751688,Stock Size: Relative,NaN,NaN,Fishing pressure: Relative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tonnes,11965.0,NaN,11965.0,http://www.ices.dk/sites/pub/Publication Repor...,13650,2020
7230,2016,Beaked redfish,reb,reb.27.5a14,169263,Sebastes mentella,27.14.a ~ 27.14.b.1 ~ 27.14.b.2 ~ 27.5.a.1 ~ 2...,0.814671,Stock Size: Relative,NaN,NaN,Fishing pressure: Relative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tonnes,9536.0,NaN,9536.0,http://www.ices.dk/sites/pub/Publication Repor...,13650,2020
7264,2012,Sole,sol,sol.27.7d,169274,Solea solea,27.7.d,1.080737,Stock Size: Relative,NaN,0.89105,Fishing pressure: Relative,NaN,1.249857,0.890635,0.6568,0.919496,0.570006,0.919496,tonnes,4131.0,NaN,4664.0,https://doi.org/10.17895/ices.advice.5947,13495,2020


In [1423]:
stock3years.columns.difference(['AssessmentYear'])

Index(['AssessmentKey', 'Blim', 'Bpa', 'Catches', 'CatchesLadingsUnits',
       'FMSY', 'FishStock', 'FishingPressure', 'FishingPressureDescription',
       'FishingPressureUnits', 'Flim', 'Fpa',
       'ICES Areas (splited with character '~')', 'Landings', 'MSYBtrigger',
       'OfficialLandings', 'Report', 'SpeciesName', 'StockKey', 'StockSize',
       'StockSizeDescription', 'StockSizeUnits', 'Year', 'enName',
       'speciesAcronym'],
      dtype='object', name=0)

In [1425]:
stock3years = pd.concat(stockList)

stock3years = stock3years.drop_duplicates(subset=['Year','FishStock'], keep='last')

,Year,enName,speciesAcronym,FishStock,StockKey,SpeciesName,ICES Areas (splited with character '~'),StockSize,StockSizeDescription,StockSizeUnits,FishingPressure,FishingPressureDescription,FishingPressureUnits,Flim,Fpa,Blim,Bpa,FMSY,MSYBtrigger,CatchesLadingsUnits,Landings,OfficialLandings,Catches,Report,AssessmentKey,AssessmentYear
596,2012,Flounder,bwq,bwq.27.2425,169097,Platichthys,27.3.d.24 ~ 27.3.d.25,94.31078,SSB,tonnes,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tonnes,10429.7322,NaN,NaN,https://doi.org/10.17895/ices.advice.6027,13363,2020
600,2016,Flounder,bwq,bwq.27.2425,169097,Platichthys,27.3.d.24 ~ 27.3.d.25,209.52005,SSB,tonnes,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tonnes,14622.31118,NaN,19779.2037824366,https://doi.org/10.17895/ices.advice.6027,13363,2020
1186,2012,Orange roughy,ory,ory.27.nea,169176,Hoplostethus atlanticus,27.1.a ~ 27.1.b ~ 27.10.a.1 ~ 27.10.a.2 ~ 27.1...,NaN,SSB,tonnes,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tonnes,185,NaN,NaN,https://doi.org/10.17895/ices.advice.5767,13446,2020
1190,2016,Orange roughy,ory,ory.27.nea,169176,Hoplostethus atlanticus,27.1.a ~ 27.1.b ~ 27.10.a.1 ~ 27.10.a.2 ~ 27.1...,NaN,SSB,tonnes,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tonnes,94,NaN,NaN,https://doi.org/10.17895/ices.advice.5767,13446,2020
1221,2012,Salmon,sal,sal.27.32,169242,Salmo salar,27.3.d.32,NaN,SSB,tonnes,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,thousands,10.89,NaN,10.89,https://doi.org/10.17895/ices.advice.5901,13453,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7414,2016,Beaked redfish,reb,reb.27.14b,169265,Sebastes mentella,27.14.b.1 ~ 27.14.b.2,1.112244,Stock Size: Relative,NaN,NaN,Fishing pressure: Relative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tonnes,NaN,NaN,3061.0,https://doi.org/10.17895/ices.advice.19772464.v1,17158,2022
7418,2020,Beaked redfish,reb,reb.27.14b,169265,Sebastes mentella,27.14.b.1 ~ 27.14.b.2,0.125017,Stock Size: Relative,NaN,NaN,Fishing pressure: Relative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tonnes,NaN,NaN,1677.0,https://doi.org/10.17895/ices.advice.19772464.v1,17158,2022
7454,2012,Beaked redfish,reb,reb.27.5a14,169263,Sebastes mentella,27.14.a ~ 27.14.b.1 ~ 27.14.b.2 ~ 27.5.a.1 ~ 2...,0.742878,Stock Size: Relative,NaN,NaN,Fishing pressure: Relative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tonnes,11965.0,NaN,11965.0,https://doi.org/10.17895/ices.advice.19772467.v1,17141,2022
7458,2016,Beaked redfish,reb,reb.27.5a14,169263,Sebastes mentella,27.14.a ~ 27.14.b.1 ~ 27.14.b.2 ~ 27.5.a.1 ~ 2...,0.805127,Stock Size: Relative,NaN,NaN,Fishing pressure: Relative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tonnes,9536.0,NaN,9536.0,https://doi.org/10.17895/ices.advice.19772467.v1,17141,2022


In [1396]:
# extract EN name and acronym of species
stock['enName'] = stock["StockDescription"].str.extract(r"^(.+?) ?(?:\d|\(|$)" , expand=False)
stock['speciesAcronym'] = stock["FishStock"].str.extract(r"^([^.]*).*" , expand=False)

In [1398]:
# check what what data is in the custom columns
# stock[~stock.CustomUnits6.isna()].dropna(axis=1)

In [1399]:
# select useful years and columns
stock = stock.loc[:,['Year','enName', 'speciesAcronym','FishStock','StockKey', 'SpeciesName', "ICES Areas (splited with character '~')", 
                     'StockSize', 'StockSizeDescription', 'StockSizeUnits',
                      'FishingPressure', 'FishingPressureDescription', 'FishingPressureUnits',
                     'Flim', 'Fpa', 'Blim', 'Bpa', 'FMSY', 'MSYBtrigger', 
                     'CatchesLadingsUnits', 'Landings', 'OfficialLandings', 'Catches',
                     'Report', 'AssessmentKey','AssessmentYear']]
stock.Year = stock.Year.astype(int)
stock = stock[stock.Year.isin([2012,2016,2020])]

In [1400]:
# To merge with OfficialCatches database
# Split the areas column into several columns and melt to get one row per species-area (as per in the CatchesOfficial) 

sepAreas = stock["ICES Areas (splited with character '~')"].str.split('~', expand=True)
sepAreas = sepAreas.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

#melt by leaving out all the columns containing the areas from "ICES Areas (splited with character '~')"
stockMelt = stock.join(sepAreas)
stockMelt = stockMelt.melt(id_vars=['Year', 'enName', 'speciesAcronym', 'FishStock', 'StockKey',
       'SpeciesName', "ICES Areas (splited with character '~')", 'StockSize',
       'StockSizeDescription', 'StockSizeUnits', 'FishingPressure',
       'FishingPressureDescription', 'FishingPressureUnits', 'Flim', 'Fpa',
       'Blim', 'Bpa', 'FMSY', 'MSYBtrigger', 'CatchesLadingsUnits', 'Landings',
       'OfficialLandings', 'Catches', 'Report', 'AssessmentKey','AssessmentYear'], 
       var_name='fullArea', value_name='area')
stockMelt = stockMelt.dropna(subset=['area'])


# (later realised could have done it with explode)

stockExplode = (stock.set_index(['Year', 'enName', 'speciesAcronym', 'FishStock', 'StockKey',
       'SpeciesName', 'StockSize',
       'StockSizeDescription', 'StockSizeUnits', 'FishingPressure',
       'FishingPressureDescription', 'FishingPressureUnits', 'Flim', 'Fpa',
       'Blim', 'Bpa', 'FMSY', 'MSYBtrigger', 'CatchesLadingsUnits', 'Landings',
       'OfficialLandings', 'Catches', 'Report','AssessmentKey','AssessmentYear'])
   .apply(lambda x: x.str.split('~').explode())
   .reset_index())  
stockExplode["ICES Areas (splited with character '~')"] = stockExplode["ICES Areas (splited with character '~')"].str.strip()

In [1401]:
# there are duplicates because of updated reports, we drop them keeping the last version
stockMelt = stockMelt.drop_duplicates(subset=['Year', 'area', 'FishStock'], keep='last')
# for consistency, do the same with the exploded version
stockExplode = stockExplode.drop_duplicates(subset=['Year', "ICES Areas (splited with character '~')", 'FishStock'], keep='last')

In [1141]:
# all ICES areas
# https://intercatch.ices.dk/CS/Data/Reports/StrataDefinitionAreaList.aspx
icesAreas = pd.DataFrame(pd.read_html('../data/icesAreas.html')[0])
icesAreas.columns = icesAreas.iloc[0]
icesAreas = icesAreas[1:]

In [1142]:
# delete parent areas to avoid double counting
# solution from https://stackoverflow.com/q/76183612/14534411
areasAll = stockMelt.area.unique().tolist()

def is_parent(p, target):
    return p.startswith(target) and len(p)>len(target) and p[len(target)] == '.'

icesAreas = []
prev = ''
for s in sorted(areasAll)+['']:
    if prev and not is_parent(s, prev):
        icesAreas.append(prev)
    prev = s

stockMelt = stockMelt[stockMelt.area.isin(icesAreas)]

In [1146]:
# ICES areas from Christoph
icesAreasC=pd.DataFrame(["27.1.a","27.1.b","27.1_NK","27.2_NK","27.2.a.1","27.2.a.2","27.2.a_NK","27.2.b.1","27.2.b.2","27.2.b_NK",
            "27.3.a.20","27.3.a.21","27.3.a_NK","27.3_NK","27.3.b.23","27.3.c.22","27.3.d_NK","27.3.d.24","27.3.d.25",
            "27.3.d.26","27.3.d.27","27.3.d.28_NK","27.3.d.29","27.3.d.30","27.3.d.31","27.3.d.32","27.3.d.28.1","27.3.d.28.2",
            "27.4.a","27.4.b","27.4.c","27.4_NK","27.5_NK","27.5.a.1","27.5.a.2","27.5.a_NK","27.5.b.2","27.5.b_NK","27.5.b.1.a",
            "27.5.b.1.b","27.5.b.1_NK","27.6.a","27.6.b_NK","27.6_NK","27.6.b.1","27.6.b.2","27.7.a","27.7.b","27.7.c.1","27.7.c.2",
            "27.7.c_NK","27.7.d","27.7.e","27.7.f","27.7.g","27.7.h","27.7.j.1","27.7.j.2","27.7.j_NK","27.7.k.1","27.7.k.2","27.7.k_NK",
            "27.8.a","27.8.b","27.8.c","27.8.d.1","27.8.d.2","27.8.d_NK","27.8.e.1","27.8.e.2","27.8.e_NK","27.8_NK","27.9.a","27.9_NK",
            "27.9.b.1","27.9.b.2","27.9.b_NK","27.10.a.1","27.10.a.2","27.10.a_NK","27.10.b","27.10_NK","27.12.a.1","27.12.a.2","27.12.a.3",
            "27.12.a.4","27.12.a_NK","27.12.b","27.12.c","27.12_NK","27.14.a","27.14.b.1","27.14.b.2","27.14.b_NK","27.14_NK","27_NK"], columns= ['icesAreas'])

# compare the areas in the stock dataset 
areaStock = pd.DataFrame(stockMelt.area.unique(), columns=['areaStock'])

# merge
areaCompare = areaStock.merge(icesAreasC, left_on='areaStock', right_on='icesAreas', how='outer', indicator=True)
# areaCompare[areaCompare['_merge']=='right_only'].reset_index(drop=True)

In [1132]:
# Wilfried's contained stock with StockSizeDescription = SSB|SSB/B45cm|B/Bmsy|Stock Size: Relative|Spawning Stock Biomass|B_index (at least for Stockassessment of 2021)
stockSSB = pd.DataFrame(stockMelt[(stockMelt.StockSizeDescription.str.contains("SSB|SSB/B45cm|B/Bmsy|Stock Size: Relative|Spawning Stock Biomass|B_index", na=False)) | (stockMelt.StockSizeDescription.isna())]
    .drop_duplicates(subset=['FishStock'], keep='last')[['FishStock', 'StockSizeDescription']]).rename(columns={'FishStock':'FishStockSSB'})

stockW = pd.DataFrame(stockW, columns=['FishStockW'])

stockCompare = stockSSB.merge(stockW, left_on='FishStockSSB', right_on='FishStockW', how='outer', indicator=True)
stockCompare[stockCompare['_merge']=='right_only'].reset_index(drop=True)

,FishStockSSB,StockSizeDescription,FishStockW,_merge
0,NaN,NaN,spr.27.4,right_only
1,NaN,NaN,reb.27.1-21,right_only
2,NaN,NaN,pok.27.6,right_only
3,NaN,NaN,hom.27.2a4a5b6a7a-ce-k8,right_only
4,NaN,NaN,cap.27.1-2,right_only
5,NaN,NaN,cod.27.1-2,right_only
6,NaN,NaN,reb.27.1-2,right_only
7,NaN,NaN,her.27.6a7bc,right_only
8,NaN,NaN,had.27.1-2,right_only
9,NaN,NaN,pra.27.1-2,right_only


In [1150]:
# keep the species by Wilfried (alternative by StockSizeDescription)
stockMelt = stockMelt[stockMelt.FishStock.isin(stockW.FishStockW)]

In [1175]:
# create dict of area-stock
areaStock = stockMelt.groupby(['speciesAcronym','area', 'FishStock']).size().reset_index().rename(columns={0:'count'})
areaStock.head()

,speciesAcronym,area,FishStock,count
0,bli,27.5.b.1.a,bli.27.5b67,3
1,bli,27.5.b.1.b,bli.27.5b67,3
2,bli,27.5.b.2,bli.27.5b67,3
3,bli,27.6.a,bli.27.5b67,3
4,bli,27.6.b.1,bli.27.5b67,3


### OfficialNominalCatches

In [1159]:
# https://www.ices.dk/data/dataset-collections/Pages/Fish-catch-and-stock-assessment.aspx
catches = pd.read_csv(("../data\OfficialNominalCatches\ICESCatchDataset2006-2020.csv")) 

In [1160]:
# add country name column
catches['geo'] = catches.Country.map(abbrev_to_country).fillna(catches.Country)

# filter countries of interest. commented as we want to compare total catches from stockAssessment 
# catches = catches[catches.geo.isin(countries)]

# convert Species to lower case
catches.Species = catches.Species.str.lower()

# 2020 has word characters, we divide it into two 
catches[['2020','2020c']] = catches['2020'].str.split(" +",expand = True) 
catches['2020'] = catches['2020'].astype(np.float64)

# keep useful columns
catches = catches.loc[:,['Country','geo','Species','Area','2012','2016','2020']]

In [1161]:
catches = catches.melt(id_vars=['Country', 'geo','Species', 'Area'], 
       var_name='Year', value_name='CatchesCountry')

In [1181]:
catchesPivot = catches.pivot_table( columns='geo', index=['Year','Area', 'Species'] , values='CatchesCountry', aggfunc='sum').reset_index()
catchesPivot['Year'] = catchesPivot['Year'].astype(int)

# only keep the areas and species that are in selected stocks
catchesPivot = catchesPivot[(catchesPivot.Area.isin(list(areaStock.area.unique()))) &\
(catchesPivot.Species.isin(list(areaStock.speciesAcronym.unique())))]

### TAC

In [1307]:
# Can be extracted from the TAC vs Advice dataset or the TAC dataset. The latter has some more rows
tac = pd.read_excel(("../data/icesTACcomparison.xlsx"), 'Council agreed TAC')
# tac = pd.read_csv(("../data/RecordOfEuropeanTAC.csv"))

In [1308]:
# extract acronym of species and convert to lower case 
# (doesn't make sense since acronym doesn't correspond to species acronyms)
# tac["speciesAcronym"] = tac["Reference"].str.extract(r"\(([\w\-]+)" , expand=False).str.lower()

In [1309]:
# filter years of interest. tac level that we care about is TAC
tac = tac[(tac.Year.isin([2012,2016,2020])) & (tac.Level == 'TAC') ]

In [1311]:
tac = tac[['Reference', 'TAC ID', 'Species', 'TAC Zone', 'Level',
       'TAC for comparison', 'Year', 'Amendment/Original']]

### ICES advice

In [1384]:
# From ICES official databse https://asd.ices.dk/AdviceList
sadOff = pd.read_csv(("../data/adviceICES_Data_26_04_2023.csv"))
# drop deprecated Advice 
sadOff = sadOff[sadOff['AdviceStatus'] == 'Advice'].copy()

In [1385]:
# transform dates
sadOff[['AdviceApplicableFrom', 'AdviceApplicableUntil']] = sadOff[['AdviceApplicableFrom', 'AdviceApplicableUntil']].apply(pd.to_datetime, format='%d/%m/%Y')

In [1386]:
# drop duplicates based on StockCode and AdviceApplicableFrom (three had the duplicates)
sadOff = sadOff.drop_duplicates(subset=['StockCode', 'AdviceApplicableFrom'], keep='last')

In [1387]:
# Create the new column with years between AdviceApplicableFrom and 
# AdviceApplicableUntil = years in which the advice is valid
date_range = lambda x: range(x['AdviceApplicableFrom'].year, x['AdviceApplicableUntil'].year+1)
sadOff = sadOff.assign(year=sadOff.apply(date_range, axis=1)).explode('year', ignore_index=True)

# keep columns of interest
sadOff = sadOff[['year', 'StockCode', 'AdviceValue', 'AdviceType', 'AdviceApplicableFrom', 'AdviceApplicableUntil',
 'AdviceValueUnit', 'AssessmentYear', 'AssessmentKey','AdviceKey', 'AdviceDOI'] ].copy()

# transform dates to year only
sadOff[['AdviceApplicableFrom', 'AdviceApplicableUntil']] = sadOff[['AdviceApplicableFrom', 'AdviceApplicableUntil']].transform(lambda x: x.dt.year) 


In [1388]:
# check stock for which advice AdviceApplicableFrom is not the year before AdviceApplicableUntil
# sadOff.loc[(sadOff['AdviceApplicableFrom']  < sadOff['AdviceApplicableUntil'] - 1)] 

In [1389]:
# In some cases, advice is given for two years, and the second year a new advice is given. 
# We drop the old advice for the second year
sadOff = sadOff.drop_duplicates(subset=['StockCode', 'year'], keep='last') 
sadOff[sadOff.duplicated(subset=['StockCode', 'year'], keep=False)] 

,year,StockCode,AdviceValue,AdviceType,AdviceApplicableFrom,AdviceApplicableUntil,AdviceValueUnit,AssessmentYear,AssessmentKey,AdviceKey,AdviceDOI


In [1390]:
# example of stock for which AdviceApplicableFrom is not the year before AdviceApplicableUntil
# sadOff[sadOff['StockCode'] == 'whg.27.6b'] 

In [ ]:
# From Carpenter 
sad = pd.read_excel(("../data/icesTACcomparison.xlsx"), 'ICES advice')
sad = sad[sad.Year.isin([2012,2016,2020])][['ICES code', 'Advice', 'Year',
        'ICES advice', 'Catches corresponding to advice',
       'Landings corresponding to advice','Choices']]

### TAC-SAC Comparison

In [ ]:
sad_tac = pd.read_excel(("../data/icesTACcomparison.xlsx"), 'Comparison') 

In [ ]:
# sad_tac[(sad_tac.Species == 'Plaice') & (sad_tac.Year.isin([2012,2016,2020])) & (sad_tac.Level == 'TAC') & (sad_tac['ICES area'] == 4)  ]

,Final Ref,Reference,TAC ID,Species,TAC Zone,ICES area,Level,TAC,Year,Amendment/Original,Sea,Amendment check,Agreement,ICES Advice,TAC above advice (t),TAC above advice (%),Net difference (t),Net difference (%),Advice change ICES,Advice change %,Advice change quantity,Previous year TAC,ICES with Council
16399,(PLE/03AN.) - 2016 - TAC - Final,(PLE/03AN.) - 2016 - TAC - Original,(PLE/03AN.),Plaice,Skagerrak,4,TAC,10056.410256,2016,Original,Atlantic,Final,Norway joint management,11108.11597,0.0,0.0,-1051.705714,-0.094679,0.766839,4.512713,-9093.11597,2015.0,11108.11597
16431,(PLE/03AN.) - 2020 - TAC - Final,(PLE/03AN.) - 2020 - TAC - Original,(PLE/03AN.),Plaice,Skagerrak,4,TAC,19647,2020,Original,Atlantic,Final,Norway joint management,19647,0.0,0.0,0.000000,0,0.170719,8.731055,-17628.00000,2019.0,19647
16808,(PLE/2A3AX4) - 2012 - TAC - Final,(PLE/2A3AX4) - 2012 - TAC - Original,(PLE/2A3AX4),Plaice,4; Union waters of 2a; that part of 3a not cov...,4,TAC,84410,2012,Original,Atlantic,Final,Norway joint management,84410,0.0,0.0,0.000000,0,0.314798,40.974142,-82399.00000,2011.0,84410
16862,(PLE/2A3AX4) - 2016 - TAC - Final,(PLE/2A3AX4) - 2016 - TAC - Original,(PLE/2A3AX4),Plaice,4; Union waters of 2a; that part of 3a not cov...,4,TAC,128376.218324,2016,Original,Atlantic,Final,Norway joint management,141801.88403,0.0,0.0,-13425.665706,-0.094679,0.104583,69.373143,-139786.88403,2015.0,141801.88403
16898,(PLE/2A3AX4) - 2020 - TAC - Final,(PLE/2A3AX4) - 2020 - TAC - Original,(PLE/2A3AX4),Plaice,4; Union waters of 2a; that part of 3a not cov...,4,TAC,146852,2020,Original,Atlantic,Final,Norway joint management,146852,0.0,0.0,0.000000,0,0.170742,71.735017,-144833.00000,2019.0,146852


## Merge Data

### Merge stockAssessment

In [1195]:
# merge stock and catches data, left or right to check non-matching rows
mStockCatch = stockMelt.merge(catchesPivot, left_on=['speciesAcronym','area','Year'], 
                            right_on=['Species', 'Area', 'Year'], how='outer', indicator='_mergeCatch')

mStockCatch['sumOffCatches'] = mStockCatch[['Belgium', 'China', 'Denmark', 'Estonia', 'Faroe Islands',
       'Finland', 'France', 'Germany', 'Greenland', 'Guernsey', 'Iceland',
       'Ireland', 'Isle of Man', 'Japan', 'Jersey', 'Korea, Republic of',
       'Latvia', 'Lithuania', 'Netherlands', 'Norway', 'Poland', 'Portugal',
       'Russian Federation', 'Spain', 'Sweden', 'Taiwan, Province of China',
       'United Kingdom of GB']].sum(axis=1)

In [1193]:
print(
len(stockMelt), 
len(catchesPivot), 
len(mStockCatch[mStockCatch._mergeCatch == 'both']),
len(mStockCatch[mStockCatch._mergeCatch == 'left_only']),
len(mStockCatch[mStockCatch._mergeCatch == 'right_only'])
)

1269 2997 1149 120 1875


In [1196]:
# Check difference between total catches from OfficialCatches and StockAssessment
mStockCatch.Catches = mStockCatch.Catches.astype(np.float64)
mStockCatch['diffCatches'] = (mStockCatch.sumOffCatches - mStockCatch.Catches)/mStockCatch.Catches
# mStockCatch.to_csv('..\dataTemp\checkCatches.csv', index=False)	
mStockCatch.to_csv('../data/stockCatch.csv')
mStockCatch['diffCatches'].describe()

count    1048.000000
mean       -0.742232
std         2.345830
min        -1.000000
25%        -1.000000
50%        -0.999504
75%        -0.931083
max        59.285888
Name: diffCatches, dtype: float64

### Merge TAC
To merge the TAC, we use the dictionary by Carpenter. 

In [1367]:
dictTAC = pd.read_excel(("../data/icesTACcomparison.xlsx"), 'Matching - ICES-TAC') 
# filter years and melt
dictTAC = dictTAC[['TAC ID', 2012, 2016, 2020]]
dictTAC = dictTAC.melt(id_vars=['TAC ID'], var_name='Year', value_name='FishStock').copy()
# explode and then merge with with original to know if TAC applies to more than one stock
dictTACexplode = dictTAC.set_index(['TAC ID', 'Year']).apply(lambda x: x.str.split('+').explode()).reset_index()
dictTACexplode = dictTACexplode.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
dictTAC = dictTAC.merge(dictTACexplode, on=['TAC ID', 'Year'])
#rename all columns of dictTAC
dictTAC.columns = ['TAC ID', 'Year', 'FishStockTAC', 'FishStock']

In [1368]:
# convert Year to float (Year in mStockCatchSADoff is float because of NaNs)
dictTAC.Year = dictTAC.Year.astype(np.float64)

In [1369]:
# filter stock from list
dictTAC = dictTAC[dictTAC.FishStock.isin(list(areaStock.FishStock.unique()))]

In [1370]:
dictTAC = dictTAC.groupby(['TAC ID', 'FishStock', 'FishStockTAC']).size().reset_index().rename(columns={0:'count'})

In [1371]:
dictTAC = dictTAC.merge(areaStock, on = 'FishStock', how='outer', indicator=True)
dictTAC.to_csv('../data/dictTACstockArea.csv')

In [1372]:
dictTAC = dictTAC[['TAC ID', 'FishStock', 'FishStockTAC','area', 'speciesAcronym']]
tacCatches =  tac.merge(dictTAC, on=['TAC ID'], how='outer', indicator=True)
# tacCatches[tacCatches._merge == 'right_only']


In [1373]:
# drop TAC without fishstock
tacCatches = tacCatches[tacCatches._merge.str.contains('right_only|both')]

In [1374]:
catchesPivot['Year'] = catchesPivot['Year'].astype(np.float64)
tacCatches = tacCatches.merge(catchesPivot, left_on=['Year', 'area','speciesAcronym'],\
 right_on=['Year', 'Area', 'Species',], how='outer', indicator='tacCatches')
tacCatches.to_csv('../data/tacCatches.csv')

If we want to merge stockAssessment - TAC - Catches:

In [169]:
# drop non-matching rows from previous merge
mStockCatch = mStockCatch[mStockCatch._mergeCatch == 'both'].copy()

mStockCatchTAC = mStockCatch.merge(dictTAC, on=['FishStock', 'Year'], how='outer', indicator='_mergeDictTAC')

In [170]:
mStockCatchTAC = mStockCatchTAC.merge(tac, on=['TAC ID', 'Year'], how='left', indicator='_mergeTAC')

In [172]:
print(
len(mStockCatch), 
len(tac), 
len(mStockCatchTAC[mStockCatchTAC._mergeDictTAC == 'both']),
len(mStockCatchTAC[mStockCatchTAC._mergeDictTAC == 'left_only']),
len(mStockCatchTAC[mStockCatchTAC._mergeDictTAC == 'right_only'])
)

2787 662 4489 1277 399


### Merge SAD (Official)

In [1394]:
# filter list of stock
sadOff = sadOff[sadOff.StockCode.isin(list(areaStock.FishStock.unique()))]
sadOff.year = sadOff.year.astype(np.float64).copy()
sadOff = sadOff[sadOff.year.isin([2012,2016,2020])]
# merge with TAC based on year and stock
sadOffTACcatches = sadOff.merge(tacCatches, left_on=['StockCode', 'year'], right_on =['FishStock', 'Year'], \
how='outer', indicator='_mergeSADTAC')
sadOffTACcatches.to_csv('../data/sadOffTACcatches.csv')

If we want to merge with stockAssessment as well

In [174]:
# drop non-matching rows from previoys merge 
mStockCatchTAC = mStockCatchTAC[mStockCatchTAC._mergeCatch == 'both'].copy()

sadOff.year = sadOff.year.astype(np.float64)

# merge with SAD using StockCode and year
mStockCatchTACSADoff = mStockCatchTAC.merge(sadOff, left_on=['FishStock','Year'],
                                             right_on=['StockCode', 'year'],
                                              how='outer', indicator='_mergeSADoff')

In [175]:
print(
len(mStockCatch), 
len(sadOff), 
len(mStockCatchTACSADoff[mStockCatchTACSADoff._mergeSADoff == 'both']),
len(mStockCatchTACSADoff[mStockCatchTACSADoff._mergeSADoff == 'left_only']),
len(mStockCatchTACSADoff[mStockCatchTACSADoff._mergeSADoff == 'right_only']),
)

2787 834 1439 4373 730


In [179]:
exportMergeAll = mStockCatchTACSADoff[(mStockCatchTACSADoff._mergeSADoff == 'both') & (mStockCatchTACSADoff._mergeTAC == 'both')]	
exportMergeAll.to_csv('..\dataTemp\StockCatchSADoffTAC.csv', index=False)

### Merge SAD (Carpenter)

In [ ]:
sad.year = sad.year.astype(np.float64)

# merge with SAD using StockCode and year
mStockCatchTACsad = mStockCatchTAC.merge(sad, left_on=['FishStock','Year'],
                                             right_on=['ICES code', 'Year'],
                                              how='outer', indicator='_mergeSAD')

In [ ]:
print(
len(mStockCatch), 
len(sadOff), 
len(mStockCatchTACSADoff[mStockCatchTACSADoff._mergeSADoff == 'both']),
len(mStockCatchTACSADoff[mStockCatchTACSADoff._mergeSADoff == 'left_only']),
len(mStockCatchTACSADoff[mStockCatchTACSADoff._mergeSADoff == 'right_only']),
)

2787 834 1439 4373 730


In [ ]:
exportMergeAll = mStockCatchTACSADoff[(mStockCatchTACSADoff._mergeSADoff == 'both') & (mStockCatchTACSADoff._mergeTAC == 'both')]	
exportMergeAll.to_csv('..\dataTemp\StockCatchSADoffTAC.csv', index=False)

## Indicators calculation

### TAC/SAD

## Trash

In [ ]:
sepAreas = stock["ICES Areas (splited with character '~')"].str.split('~', expand=True)
sepAreas = sepAreas.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [ ]:
list1 = stock["ICES Areas (splited with character '~')"].to_list()

newList= []
for item in list1:
    if(str(item) != "nan"):
        newList.append(item)

max_len = -1
for ele in newList: 
    if(len(ele) > max_len): 
        max_len = len(ele) 
        res = ele 
        
print("Longest String is : ", res) 

Longest String is :  27.1.a ~ 27.1.b ~ 27.10.a.1 ~ 27.10.a.2 ~ 27.10.b ~ 27.12.a.1 ~ 27.12.a.2 ~ 27.12.a.3 ~ 27.12.a.4 ~ 27.12.b ~ 27.12.c ~ 27.14.a ~ 27.14.b.1 ~ 27.14.b.2 ~ 27.2.a.1 ~ 27.2.a.2 ~ 27.2.b.1 ~ 27.2.b.2 ~ 27.3.a ~ 27.3.b.23 ~ 27.3.c.22 ~ 27.3.d.24 ~ 27.3.d.25 ~ 27.3.d.26 ~ 27.3.d.27 ~ 27.3.d.28.1 ~ 27.3.d.28.2 ~ 27.3.d.29 ~ 27.3.d.30 ~ 27.3.d.31 ~ 27.3.d.32 ~ 27.4.a ~ 27.4.b ~ 27.4.c ~ 27.5.a.1 ~ 27.5.a.2 ~ 27.5.b.1.a ~ 27.5.b.1.b ~ 27.5.b.2 ~ 27.6.a ~ 27.6.b.1 ~ 27.6.b.2 ~ 27.7.a ~ 27.7.b ~ 27.7.c.1 ~ 27.7.c.2 ~ 27.7.d ~ 27.7.e ~ 27.7.f ~ 27.7.g ~ 27.7.h ~ 27.7.j.1 ~ 27.7.j.2 ~ 27.7.k.1 ~ 27.7.k.2 ~ 27.8.a ~ 27.8.b ~ 27.8.c ~ 27.8.d.1 ~ 27.8.d.2 ~ 27.8.e.1 ~ 27.8.e.2 ~ 27.9.a ~ 27.9.b.1 ~ 27.9.b.2


In [ ]:
pleStock = stock[stock.FishStock == 'san.sa.1r']
pleStock = pleStock["ICES Areas (splited with character '~')"].str.split('~', expand=True).iloc[0,:].to_list()
pleStock = [s.strip() for s in pleStock]

In [ ]:
denmarkPle = catches[(catches['Area'].isin(pleStock))  & (catches['Species'] == 'san')].copy()
denmarkPle['2020'] = denmarkPle['2020'].astype(np.float64)
denmarkPle

KeyError: '2020'